In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## train data 불러오기

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

### 일반 전처리
- num_songs,num_tags = '중복제거한 노래 태그 개수'

In [3]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [4]:
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

In [5]:
def list_merger(x):
    temp_list.extend(x)

temp_list = []

train_df['songs'].map(list_merger)

unique_songs = list(set(temp_list))

num_songs = len(unique_songs)

In [6]:
temp_list = []
                
train_df['tags'].map(list_merger)
                    
unique_tags = list(set(temp_list))
                
num_tags = len(unique_tags)

In [7]:
num_songs

615142

In [8]:
num_tags

29160

In [9]:
num_train = len(train_df)
num_train

115071

### tag에 id부여

In [10]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28


In [11]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = {}


# { 새로운id : 태그 } 딕셔너리
id_to_tag = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_tags):
    id_to_tag[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
tag_to_id = {v:k for k,v in id_to_tag.items()}


# tags를 id로 변환해서 새로운 column을 생성
def tag_id_trans(x):
    
    temp_list = []
    
    for tag in x:
        temp_list.append(tag_to_id[tag])
    
    return temp_list

train_df['tags_id'] = train_df['tags'].map(tag_id_trans)

In [12]:
train_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[16846]
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[27312, 18129]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[19601, 615]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38,"[6914, 25798, 26081, 11925, 25092, 18983, 1646..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53,[13344]


### 플레이리스트 속 노래들(songs)에게 새로운 id 부여

In [13]:
# { 노래id : 새로운id } 딕셔너리
song_to_id = {}


# { 새로운id : 노래id } 딕셔너리
id_to_song = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_songs):
    id_to_song[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
song_to_id = {v:k for k,v in id_to_song.items()}


# tags를 id로 변환해서 새로운 column을 생성
def song_id_trans(x):
    
    temp_list = []
    
    for song in x:
        temp_list.append(song_to_id[song])
    
    return temp_list

train_df['new_songs_id'] = train_df['songs'].map(song_id_trans)

In [14]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[16846],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[27312, 18129]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[19601, 615]","[72132, 240434, 144495, 161861, 307991, 222934..."


### 플레이리스트 새로운 id 부여

In [15]:
train_df['new_plyst_id'] = train_df.index

In [16]:
id_to_playlist = {}

playlist_to_id = {}

def mk_dict(x):
    
    id_to_playlist[x['new_plyst_id']] = x['id']

train_df.loc[:,['id','new_plyst_id']].apply(mk_dict,axis=1)

playlist_to_id = {v:k for k,v in id_to_playlist.items()}

In [17]:
train_df = train_df[['tags','tags_id','id','new_plyst_id','plylst_title','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,tags,tags_id,id,new_plyst_id,plylst_title,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,[락],[16846],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[27312, 18129]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[19601, 615]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [18]:
train_df.columns = ['태그','태그_id','플리_id','새플리_id','플리제목','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,[락],[16846],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[27312, 18129]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[19601, 615]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

<현재>

- num_songs : 중복 제거된 총 노래수 615142
- num_tags : 중복 제거된 총 태그수 29160
- num_train : 데이터의 총 갯수 115071
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리
- playlist_to_id { 플리id : 새로운id } 딕셔너리 
- id_to_playlist { 새로운id : 플리id } 딕셔너리 

<이후>
- code_to_genre {장르코드: 장르} 딕셔너리
- genre_to_code {장르 : 장르코드} 딕셔너리

# 장르와 코드로 협업필터링 추천해보자

## genre 앞의 네글자로 대장르만 가져오기

In [19]:
genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

# 찍어보기
#genre_big

## 세부장르에 대분류 장르의 이름 넣기

In [20]:
code_to_genre = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        code_to_genre[k] = new_value
        
    # 아니면 그대로
    else:
        code_to_genre[k] = v

# 찍어보기
#genre_new

## 장르 딕셔너리 키 밸류 바꾸기 -> 나중에 장르 이름으로 코드 찾을 거기 때문임!

In [22]:
genre_to_code = {v:k for k,v in code_to_genre.items()}
# 찍어보기
#genre_to_code 

## Train과 장르 합치기
- 플레이리스트 안에 들어있는 노래의 장르를 중복제거 해서 컬럼에 추가
- 35초 정도 걸렸음

In [23]:
def code_gen(x):
    
    # 모든 장르를 담을 리스트
    temp_list = []
       
    # 플레이리스트에 들어있는 장르를 extend로 싸그리 모아줌
    # song노래의 id는 index와 같다는 점을 참고!
    for id in x:
        temp_list.extend(song_df.loc[id,'song_gn_dtl_gnr_basket'])
        
    # 중복제거
    temp_list = list(set(temp_list))
    
    return temp_list

train_df['장르'] = train_df['노래_id'].map(code_gen)
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일,장르
0,[락],[16846],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000,"[GN1013, GN0904, GN1101, GN1301, GN1003, GN100..."
1,"[추억, 회상]","[27312, 18129]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000,"[GN0501, GN0101, GN1504, GN0901, GN0805, GN100..."
2,"[까페, 잔잔한]","[19601, 615]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000,"[GN0403, GN0501, GN0303, GN0101, GN0301, GN040..."


## 태그_id 하나당 어떤 장르가 포함되었는지 araboja

- 각 태그가 몇번 출몰했는지 알아야함
- 

In [27]:
tag_all = []

def collect_tag(x):
    global tag_all
    
    tag_all.extend(x)
    
train_df['태그_id'].map(collect_tag)

tag_unique = list(set(tag_all))

len(tag_unique)

29160

In [33]:
from collections import Counter

tag_count = dict(Counter(tag_all))

tag_count

{16846: 4007,
 27312: 6520,
 18129: 4579,
 19601: 2856,
 615: 10218,
 6914: 468,
 25798: 108,
 26081: 75,
 11925: 2467,
 25092: 112,
 18983: 7,
 16468: 208,
 26687: 948,
 1580: 34,
 10759: 6,
 13344: 3293,
 5193: 2646,
 24122: 10095,
 7515: 3248,
 19609: 73,
 25588: 9783,
 10705: 16465,
 16011: 105,
 16613: 6425,
 4386: 132,
 23310: 2234,
 23138: 272,
 24216: 1352,
 18473: 3527,
 22505: 497,
 1835: 8704,
 9507: 5400,
 19101: 986,
 26165: 345,
 22216: 327,
 22881: 4531,
 1699: 795,
 9261: 94,
 13766: 1,
 22978: 125,
 15213: 630,
 17504: 309,
 16109: 13,
 20869: 212,
 24192: 194,
 27796: 4559,
 22150: 4,
 13333: 14,
 8283: 9,
 22001: 8,
 3748: 7,
 19931: 9,
 6876: 7,
 25655: 4,
 7540: 1133,
 8774: 1,
 27415: 130,
 27449: 6413,
 426: 792,
 14324: 7615,
 3408: 8320,
 18128: 996,
 25268: 871,
 8155: 272,
 11843: 46,
 14713: 4042,
 22690: 3213,
 5235: 11417,
 2652: 18,
 28852: 148,
 28428: 3387,
 6381: 4910,
 21998: 2982,
 18567: 1,
 11680: 13,
 1552: 3,
 10317: 4,
 12865: 19,
 24722: 100,
 